In [1]:
import numpy as np
from utility import *

In [4]:
def gen_qbham(h, nqb = None):

    I, X, Y, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[0, -1j], [1j, 0]]), np.array([[1, 0], [0, -1]])

    # Build matrix representiation of the Hamiltonian H
    if(nqb is None):
        n_qubits = openfermion.count_qubits(h)
    else:
        n_qubits = nqb   

    h_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
    for term, term_coeff in h.terms.items(): # Iterate over pauli-words of H
        term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']

        # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
        pw_matrix = np.identity(1)
        for i in range(n_qubits):
            if i not in term:        pw_matrix = np.kron(pw_matrix, I)
            else:
                if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
                elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
                else:                pw_matrix = np.kron(pw_matrix, Z)
        h_matrix += pw_matrix * term_coeff

    return h_matrix


def op_var(op, wf):
    
    op2 = np.dot(op, op)
    var = np.dot(np.dot(wf.conj(),op2),wf) - np.dot(np.dot(wf.conj(),op),wf)**2
    
    return var

In [6]:
h = get_qubit_hamiltonian(mol='h2', geometry=0.741, basis='sto3g', qubit_transf='jw')
n_qubits = openfermion.count_qubits(h)
print('Number of qubits needed:',n_qubits)

h_m = gen_qbham(h) 
evals, evecs = np.linalg.eigh(h_m) 
psi = evecs[:,0]
var_h = op_var(h_m, psi)
print('H variance: ', var_h)

Number of qubits needed: 4
H variance:  (-4.440892098500626e-16+0j)


In [7]:
etol = 1.0e-3

#Error estimate from qubit-wise commuting partitioning
qwc_list = get_qwc_group(h)    
var_qwc = []
for qwc in qwc_list:
    uz = get_zform_unitary(qwc) #unitary rotation for z-gate measurement
    allz = remove_complex(uz * qwc * uz)
       
    uhn = gen_qbham(allz, n_qubits)
    var_hn = op_var(uhn, psi)
    var_qwc.append(var_hn**0.5)

print('QWC N_T:', (np.sum(var_qwc)/etol)**2)
    
#Error estimate from fully commuting partitioning
comm_groups = get_commuting_group(h) 
var_fc = []
for hn in comm_groups.values():
    uqwc = get_qwc_unitary(hn)
    qwc = remove_complex(uqwc * hn * uqwc)
    
    uz = get_zform_unitary(qwc) #unitary rotation for z-gate measurement
    allz = remove_complex(uz * qwc * uz)
    
    uhn = gen_qbham(allz, n_qubits)
    var_hn = op_var(uhn, psi)
    var_fc.append(var_hn**0.5)
    
print('FC N_T:', (np.sum(var_fc)/etol)**2)

QWC N_T: (31208.047958562664+0j)
FC N_T: (32929.34741061861+0j)
